<a href="https://colab.research.google.com/github/zeph0yr/Operation-Research-Optimization/blob/master/Travelling%20Salesman%20Problem_Lab11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

     |████████████████████████████████| 40.6MB 104kB/s 


In [ ]:
from pulp import *
import numpy as np
import pandas as pd

#Question 1



##Import .csv file

In [ ]:
from google.colab import files
data=files.upload()
for x in data.keys():
  print(f"User uploaded file {x} with size {len(data[x])} bytes")

Saving Lab11.csv to Lab11 (2).csv
User uploaded file Lab11.csv with size 113 bytes


##Printing .csv file

In [ ]:
df=pd.read_csv('Lab11.csv')
df

,1,2,3,4,5,6
0,0,10,100,50,33,66
1,10,0,22,86,952,3
2,100,22,0,6,86,2
3,50,86,6,0,5,4
4,33,952,86,5,0,9
5,66,3,2,4,9,0


In [ ]:
nvar=6*6
city=['1','2','3','4','5','6']
dist=df.iloc[0:6,0:6].to_numpy()
cost = dict(((a,b),dist[ord(a)-48-1][ord(b)-48-1]) for a in city for b in city if a!=b )

##Define lpp problem

In [ ]:
model=LpProblem("lab11",LpMinimize)

##Define decision variables

In [ ]:
x = LpVariable.dicts('x',cost, 0,1,LpBinary)

##Define objective function

In [ ]:
model+=lpSum([x[(i,j)]*cost[(i,j)] for (i,j) in cost])


##Add constraints

In [ ]:
for k in city:
    model+= lpSum([ x[(i,k)] for i in city if (i,k) in x]) ==1 ##i to j
    model+=lpSum([ x[(k,i)] for i in city if (k,i) in x]) ==1  ##j to i

##Linking Subtours

In [ ]:
u = LpVariable.dicts('u', city, 0, len(city)-1, LpInteger)
for i in city:
    for j in city:
        if i != j and (i != '1' and j!= '1') and (i,j) in x:
            model += u[i] - u[j] <= (6)*(1-x[(i,j)]) - 1
model

lab11:
MINIMIZE
10*x_('1',_'2') + 100*x_('1',_'3') + 50*x_('1',_'4') + 33*x_('1',_'5') + 66*x_('1',_'6') + 10*x_('2',_'1') + 22*x_('2',_'3') + 86*x_('2',_'4') + 952*x_('2',_'5') + 3*x_('2',_'6') + 100*x_('3',_'1') + 22*x_('3',_'2') + 6*x_('3',_'4') + 86*x_('3',_'5') + 2*x_('3',_'6') + 50*x_('4',_'1') + 86*x_('4',_'2') + 6*x_('4',_'3') + 5*x_('4',_'5') + 4*x_('4',_'6') + 33*x_('5',_'1') + 952*x_('5',_'2') + 86*x_('5',_'3') + 5*x_('5',_'4') + 9*x_('5',_'6') + 66*x_('6',_'1') + 3*x_('6',_'2') + 2*x_('6',_'3') + 4*x_('6',_'4') + 9*x_('6',_'5') + 0
SUBJECT TO
_C1: x_('2',_'1') + x_('3',_'1') + x_('4',_'1') + x_('5',_'1') + x_('6',_'1')
 = 1

_C2: x_('1',_'2') + x_('1',_'3') + x_('1',_'4') + x_('1',_'5') + x_('1',_'6')
 = 1

_C3: x_('1',_'2') + x_('3',_'2') + x_('4',_'2') + x_('5',_'2') + x_('6',_'2')
 = 1

_C4: x_('2',_'1') + x_('2',_'3') + x_('2',_'4') + x_('2',_'5') + x_('2',_'6')
 = 1

_C5: x_('1',_'3') + x_('2',_'3') + x_('4',_'3') + x_('5',_'3') + x_('6',_'3')
 = 1

_C6: x_('3',_'1') +

##Solve

In [ ]:
status=model.solve()

print(f"status: {model.status},{LpStatus[model.status]}")
print(f"Objective= {model.objective.value()}")

for var in x.values():
  print("{}={}".format(var.name,var.value()))

status: 1,Optimal
Objective= 59.0
x_('1',_'2')=0.0
x_('1',_'3')=0.0
x_('1',_'4')=0.0
x_('1',_'5')=1.0
x_('1',_'6')=0.0
x_('2',_'1')=1.0
x_('2',_'3')=0.0
x_('2',_'4')=0.0
x_('2',_'5')=0.0
x_('2',_'6')=0.0
x_('3',_'1')=0.0
x_('3',_'2')=0.0
x_('3',_'4')=0.0
x_('3',_'5')=0.0
x_('3',_'6')=1.0
x_('4',_'1')=0.0
x_('4',_'2')=0.0
x_('4',_'3')=1.0
x_('4',_'5')=0.0
x_('4',_'6')=0.0
x_('5',_'1')=0.0
x_('5',_'2')=0.0
x_('5',_'3')=0.0
x_('5',_'4')=1.0
x_('5',_'6')=0.0
x_('6',_'1')=0.0
x_('6',_'2')=1.0
x_('6',_'3')=0.0
x_('6',_'4')=0.0
x_('6',_'5')=0.0


##Get the results

In [ ]:
upd = '1'
shortest_route=[]
shortest_route.append(city.pop(0))

while len(city) > 0:

    for k in city:
        if x[(upd,k)].varValue ==1:
            shortest_route.append( city.pop( city.index(k)))
            upd=k
            break

shortest_route.append('1')

shortest_route_length = [cost[(shortest_route[i-1], shortest_route[i])] for i in range(1,len(shortest_route))]

print('Shortest Path is:')
print(' -> '.join(shortest_route))
print(f"Minimum cost of the tour is :{sum(shortest_route_length)}")

Shortest Path is:
1 -> 5 -> 4 -> 3 -> 6 -> 2 -> 1
Minimum cost of the tour is :59
